In [1]:
import pandas as pd

In [16]:
df=pd.read_csv('D:\weibo_date\date_4w.csv', encoding='gb18030')

In [17]:
x = df[['comment']]
y = df.sentiment

In [18]:
import jieba

In [19]:
def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))

In [20]:
x['cutted_comment'] = x.comment.apply(chinese_word_cut)

In [21]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1)

In [22]:
def get_custom_stopwords(stop_words_file):
    with open(stop_words_file) as f:
        stopwords = f.read()
    stopwords_list = stopwords.split('\n')
    custom_stopwords_list = [i for i in stopwords_list]
    return custom_stopwords_list

In [23]:
stop_words_file = "stopwordsHIT.txt"
stopwords = get_custom_stopwords(stop_words_file)

In [24]:
''''#  采用普通统计CountVectorizer提取特征向量
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer( max_df = 0.8,
                       min_df = 3,
                       token_pattern=u'[\u4e00-\u9fa5]+',#u'(?u)\\b[^\\d\\W]\\w+\\b',#u'(?u)\b\w+\b
                       #stop_words=frozenset(stopwords),
                       ngram_range=(1,1))'''

"'#  采用普通统计CountVectorizer提取特征向量\nfrom sklearn.feature_extraction.text import CountVectorizer\nvec = CountVectorizer( max_df = 0.8,\n                       min_df = 3,\n                       token_pattern=u'[一-龥]+',#u'(?u)\\b[^\\d\\W]\\w+\\b',#u'(?u)\x08\\w+\x08\n                       #stop_words=frozenset(stopwords),\n                       ngram_range=(1,1))"

In [25]:
#   采用TfidfVectorizer提取文本特征向量
from sklearn.feature_extraction.text import  TfidfVectorizer
tfid_vec = TfidfVectorizer(max_df = 0.8,
                       min_df = 3,
                       token_pattern=u'[\u4e00-\u9fa5]+',#u'(?u)\\b[^\\d\\W]\\w+\\b',
                       #stop_words=frozenset(stopwords),
                       ngram_range=(1,1))

In [26]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None)

In [27]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(tfid_vec, nb)

In [28]:
pipe.fit(x_train.cutted_comment, y_train)

Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.8, max_features=None, min_df=3,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth...   vocabulary=None)), ('multinomialnb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [29]:
x_tfid_test = tfid_vec.transform(x_test.cutted_comment)
nb.score(x_tfid_test, y_test)

0.8638